In [2]:
import pandas as pd
import sqlite3

In [3]:
con = sqlite3.connect(r"data\rls_db\rls.sqlite")

In [4]:
df = pd.read_sql_query('''
    SELECT 
    *  
    FROM drugs dr
    JOIN tradenames tn ON dr.tradename_id = tn.id
    JOIN descriptions d ON dr.xnn_id = d.object_id 
    join (SELECT
        object_type,
        f1 AS df1,
        f2 AS df2,
        f3 AS df3,
        f4 AS df4,
        f5 AS df5,
        f6 AS df6,
        f7 AS df7,
        f8 AS df8,
        f9 AS df9,
        f10 AS df10,
        f11 AS df11,
        f12 AS df12,
        f13 AS df13,
        f14 AS df14,
        f15 AS df15,
        f16 AS df16,
        f17 AS df17,
        f18 AS df18,
        f19 AS df19,
        f20 AS df20,
        f21 AS df21,
        f22 AS df22,
        f23 AS df23,
        f24 AS df24,
        f25 AS df25,
        f26 AS df26,
        f27 AS df27,
        f28 AS df28
    FROM descriptions
    WHERE object_id < 0) d_t ON d_t.object_type = d.object_type
    group by tradename_id
''', con)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Columns: 133 entries, id to df28
dtypes: float64(8), int64(58), object(67)
memory usage: 352.4+ KB


In [5]:
con.close()

In [6]:
print(df.head(2))

       id  tradename_id  latname_id  aphs_id  dosageform_id  \
0  532543           435       445.0    151.0              4   
1    2162           479       476.0    261.0             59   

   dosageformchars_id  ntfr_id  firm_id  customer_id      dosage  ...  \
0                 NaN        4    14378        14812      500 мг  ...   
1                 NaN        4    11842          431  0.05%+0.1%  ...   

            df19 df20  df21           df22        df23         df24  \
0  Форма выпуска             Производитель  Литература  Комментарий   
1  Форма выпуска             Производитель  Литература  Комментарий   

         df25                      df26  df27  df28  
0  Инструкция  Условия отпуска из аптек              
1  Инструкция  Условия отпуска из аптек              

[2 rows x 133 columns]


In [7]:
drugs = df.T.to_dict().values()

In [8]:
def drugToHTML(drug):
    htmlItem = f'''
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>{drug[f'name']}</title>
    </head>
    <body>
        <h1>{drug[f'name']}</h1>
    </body>
    </html>
    '''
    for i in range(1, 29):
        if drug[f"f{i}"]:
            htmlItem += f'''
            <div>
                <h2> {drug[f"df{i}"]} </h2>
                {drug[f"f{i}"]}
            </div>
            '''
    htmlItem += '''
    </body>
    </html>
    '''
    return htmlItem

In [9]:
drugsDocs = [{'name': drug['name'], 'html': drugToHTML(drug)} for drug in drugs]

In [10]:
import weasyprint

In [13]:
for drugDoc in drugsDocs:
    weasyprint.HTML(string=drugDoc['html']).write_pdf(f'data//rls//{drugDoc["name"]}.pdf')